In [1]:
import yfinance  as yf
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
import numpy as np
import pandas as pd

In [2]:
start=dt.datetime(2022,7,1)
end=dt.datetime(2023,7,28)
symbol='SBIN.NS'

In [3]:
stk_data = yf.download(symbol, start=start, end=end)

[*********************100%%**********************]  1 of 1 completed


In [4]:
stk_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-07-01,463.350006,468.799988,460.549988,466.850006,457.950867,9405495
2022-07-04,466.850006,474.149994,466.850006,473.450012,464.425049,9186358
2022-07-05,475.149994,480.000000,471.000000,472.149994,463.149811,12003244
2022-07-06,475.149994,481.000000,473.899994,479.950012,470.801147,12059857
2022-07-07,484.000000,488.250000,481.600006,486.750000,477.471527,11497832
...,...,...,...,...,...,...
2023-07-21,610.000000,619.500000,609.000000,615.099976,615.099976,21040281
2023-07-24,615.049988,621.900024,614.299988,617.650024,617.650024,11756364
2023-07-25,621.799988,621.799988,604.200012,609.599976,609.599976,18503305


In [5]:
stk_data=stk_data[['Open','High','Low','Close']]

In [6]:
stk_data

,Open,High,Low,Close
Date,,,,
2022-07-01,463.350006,468.799988,460.549988,466.850006
2022-07-04,466.850006,474.149994,466.850006,473.450012
2022-07-05,475.149994,480.000000,471.000000,472.149994
2022-07-06,475.149994,481.000000,473.899994,479.950012
2022-07-07,484.000000,488.250000,481.600006,486.750000
...,...,...,...,...
2023-07-21,610.000000,619.500000,609.000000,615.099976
2023-07-24,615.049988,621.900024,614.299988,617.650024
2023-07-25,621.799988,621.799988,604.200012,609.599976


In [7]:
column='Open'

In [8]:
series =stk_data['Open'].values.reshape(-1, 1)

In [9]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(series)

In [10]:
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        label = data[i + seq_length]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [11]:
sequence_length = 10

In [12]:
X, y = create_sequences(scaled_data, sequence_length)

In [13]:
X = X.reshape((X.shape[0], X.shape[1], 1))

In [14]:
model = Sequential()
model.add(GRU(50, activation='relu', input_shape=(sequence_length, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse') 


In [15]:
model.fit(X, y, epochs=50, batch_size=32, validation_split=0.1)

Epoch 1/50

8/8 [==============================] - 2s 56ms/step - loss: 0.3252 - val_loss: 0.4022
Epoch 2/50
8/8 [==============================] - 0s 11ms/step - loss: 0.2261 - val_loss: 0.2863
Epoch 3/50
8/8 [==============================] - 0s 11ms/step - loss: 0.1526 - val_loss: 0.1886
Epoch 4/50
8/8 [==============================] - 0s 10ms/step - loss: 0.0933 - val_loss: 0.1101
Epoch 5/50
8/8 [==============================] - 0s 9ms/step - loss: 0.0447 - val_loss: 0.0417
Epoch 6/50
8/8 [==============================] - 0s 9ms/step - loss: 0.0146 - val_loss: 0.0065
Epoch 7/50
8/8 [==============================] - 0s 9ms/step - loss: 0.0143 - val_loss: 0.0042
Epoch 8/50
8/8 [==============================] - 0s 9ms/step - loss: 0.0121 - val_loss: 0.0093
Epoch 9/50
8/8 [==============================] - 0s 10ms/step - loss: 0.0102 - val_loss: 0.0139
Epoch 10/50
8/8 [==============================] - 0s 9ms/step - loss: 0.0090 - val_loss: 0.0083
Epoch 11/50
8/8 [================

In [16]:
test_data = scaled_data[-sequence_length:]  
test_data = test_data.reshape((1, sequence_length, 1))

In [17]:
predicted_value = model.predict(test_data)
predicted_value = scaler.inverse_transform(predicted_value)

1/1 [==============================] - 0s 258ms/step


In [18]:
print("Predicted Value:", predicted_value[0, 0])

Predicted Value: 613.95404


In [19]:
actual_value = series[-1, 0]
print("Actual Value:", actual_value)

Actual Value: 616.4000244140625


In [20]:
actual_value_arrary=np.array([actual_value])

In [21]:
predicted_value_array = np.array([predicted_value[0, 0]])

In [22]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(actual_value_arrary , predicted_value_array)
print("Mean Squared Error (MSE):", mse)

Mean Squared Error (MSE): 5.982837174087763
